#### Author: Faolán Hamilton

## Part 1 70%
Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.

- Weighted mean age (by sex)
- The difference between the sexes by age
This part does not need to look at the regions.

i.e. You can take the notebook I used in the lectures and substitute the sexes for the regions.

## Part 2 20%
- In the same notebook, make a variable that stores an age (say 35).

- Write that code that would group the people within 5 years of that age together, into one age group 

- Calculate the population difference between the sexes in that age group.

## Part 3 10%
In the same notebook.

- Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

--------------------

## Part 1
- This notebook is for assignment 5 of the Programming for Data Analytics module. This structure has been edited from the 'weighted stats' notebook created by Andrew Beatty


In [47]:
# The first step is bringing in modules used in this notebook
import pandas as pd
import numpy as np

In [2]:
# This is the dataset with the ages broken down by sex in Ireland
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
#Read in the url as a csv
df = pd.read_csv(url)
#Show the last three rows of data
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


The data currently list out all Local Authorities in the output. For the first section, I want to create a subset of this dataset using only Ireland level data

In [3]:
#We only need data for Ireland, so the other rows can be removed for now
df = df[df["Administrative Counties"] == "Ireland"]
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9696,FY006AC01,Population,2022,2022,2,Female,098,98 years,IE0,Ireland,Number,492
9728,FY006AC01,Population,2022,2022,2,Female,099,99 years,IE0,Ireland,Number,336
9760,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,IE0,Ireland,Number,584


There are a lot of columns in the dataset, but we only need 'Sex', Single Year of Age' and 'VALUE'

In [4]:
# Show all the columns in the dataset to identify which to remove
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [5]:
#remove the headers that are not relevant
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)
df

,Sex,Single Year of Age,Administrative Counties,VALUE
0,Both sexes,All ages,Ireland,5149139
32,Both sexes,Under 1 year,Ireland,57796
64,Both sexes,1 year,Ireland,56420
96,Both sexes,2 years,Ireland,59210
128,Both sexes,3 years,Ireland,60484
...,...,...,...,...
9632,Female,96 years,Ireland,956
9664,Female,97 years,Ireland,732
9696,Female,98 years,Ireland,492
9728,Female,99 years,Ireland,336


In [6]:
#improve the naming convention of the 'Single Year of Age'
df = df[df["Single Year of Age"] != "All ages"]
#Change 'Under 1 year' to '0'
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\\D', '', regex=True)
df

C:\Users\ClaireReilly\AppData\Local\Temp\ipykernel_19532\2402347282.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
C:\Users\ClaireReilly\AppData\Local\Temp\ipykernel_19532\2402347282.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Single Year of Age'] = df['Single Year of Age'].str.replace('\\D', '', regex=True)


,Sex,Single Year of Age,Administrative Counties,VALUE
32,Both sexes,0,Ireland,57796
64,Both sexes,1,Ireland,56420
96,Both sexes,2,Ireland,59210
128,Both sexes,3,Ireland,60484
160,Both sexes,4,Ireland,61505
...,...,...,...,...
9632,Female,96,Ireland,956
9664,Female,97,Ireland,732
9696,Female,98,Ireland,492
9728,Female,99,Ireland,336


In [7]:
#Ensure 'Single Year of Age' and 'VALUE' are integers
df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')
#print (df.head(3))
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 303 entries, 32 to 9760
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      303 non-null    object
 1   Single Year of Age       303 non-null    int64 
 2   Administrative Counties  303 non-null    object
 3   VALUE                    303 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 11.8+ KB


C:\Users\ClaireReilly\AppData\Local\Temp\ipykernel_19532\3479022079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Single Year of Age']=df['Single Year of Age'].astype('int64')
C:\Users\ClaireReilly\AppData\Local\Temp\ipykernel_19532\3479022079.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VALUE']=df['VALUE'].astype('int64')


,Sex,Single Year of Age,Administrative Counties,VALUE
32,Both sexes,0,Ireland,57796
64,Both sexes,1,Ireland,56420
96,Both sexes,2,Ireland,59210
128,Both sexes,3,Ireland,60484
160,Both sexes,4,Ireland,61505
...,...,...,...,...
9632,Female,96,Ireland,956
9664,Female,97,Ireland,732
9696,Female,98,Ireland,492
9728,Female,99,Ireland,336


I wanted to ensure the csv output values were rounded to whole values and found this Stackoverflow (https://stackoverflow.com/questions/76584654/python-pandas-to-csv-is-rounding-any-way-for-it-to-not-round)

In [8]:
df_analysis = pd.pivot_table(df,'VALUE',"Single Year of Age", 'Sex')
# write out the entire file to local machine
df_analysis.to_csv(f"assignment05_population_for_analysis.csv", float_format='%.0f')
print (df_analysis.head(3))

Sex                 Both sexes   Female     Male
Single Year of Age                              
0                      57796.0  28186.0  29610.0
1                      56420.0  27545.0  28875.0
2                      59210.0  28974.0  30236.0


---------------------------------------------------------------

## Weighted mean age by sex

In [34]:
headers = list(df_analysis.columns)
female = headers[1]
male = headers[2]

In [ ]:
female_weighted_mean = np.average(df_analysis.index, weights=df_analysis[female])
female_rounded = round(female_weighted_mean,2)
print (f'The weighted mean for the female population is {female_rounded}')

The weighted mean for the female population is 38.94


In [48]:
male_weighted_mean = np.average(df_analysis.index, weights=df_analysis[male])
male_rounded = round(male_weighted_mean,2)
print (f'The weighted mean for the male population is {male_rounded}')

The weighted mean for the male population is 37.74


## The difference between the sexes by age

In [56]:
# Calculating the difference between sex
sex_difference = df_analysis[female]-df_analysis[male]
sex_difference

Single Year of Age
0     -1424.0
1     -1330.0
2     -1262.0
3     -1518.0
4     -1867.0
        ...  
96      629.0
97      515.0
98      362.0
99      231.0
100     430.0
Length: 101, dtype: float64

In [57]:
# adding the sex_difference to the .csv 
df_analysis['Difference'] = sex_difference
df_analysis.to_csv(f"assignment05_population_for_analysis.csv", float_format='%.0f')
print (df_analysis.head(3))

Sex                 Both sexes   Female     Male  Difference
Single Year of Age                                          
0                      57796.0  28186.0  29610.0     -1424.0
1                      56420.0  27545.0  28875.0     -1330.0
2                      59210.0  28974.0  30236.0     -1262.0


## Make a variable that stores an age (say 35).

In [86]:
age_35 = df_analysis.loc[35]
age_35

Sex
Both sexes    72635.0
Female        37940.0
Male          34695.0
Difference     3245.0
Name: 35, dtype: float64

## Write that code that would group the people within 5 years of that age together, into one age group 

In [96]:
filtered_df = df_analysis[df_analysis.index > 90]

TypeError: '>' not supported between instances of 'str' and 'int'

In [93]:
df_analysis[df_analysis$index > 10 & df_analysis$index > 5]

SyntaxError: invalid syntax (220626089.py, line 1)

In [92]:
df_analysis.index

Index([                   0,                    1,                    2,
                          3,                    4,                    5,
                          6,                    7,                    8,
                          9,
       ...
                         92,                   93,                   94,
                         95,                   96,                   97,
                         98,                   99,                  100,
       'Single Year of Age'],
      dtype='object', name='Single Year of Age', length=102)

In [91]:
filtered_rows = df_analysis.loc[df_analysis.index > 25]

TypeError: '>' not supported between instances of 'str' and 'int'